In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile
from six.moves import urllib

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string("model_dir", "", "Base directory for output models.")
flags.DEFINE_string("model_type", "wide_n_deep",
                    "Valid model types: {'wide', 'deep', 'wide_n_deep'}.")
flags.DEFINE_integer("train_steps", 200, "Number of training steps.")

In [6]:
#all numbers changed with letter because python does NOT like numbers at the start of variables...
COLUMNS = ["MSSubClass", "MSZoning", "LotFrontage", "LotArea", "Street", "Alley",
           "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope","Neighborhood",
           "Condition1", "Condition2", "BldgType", "HouseStyle", "OverallQual", "OverallCond",
           "YearBuilt", "YearRemodAdd", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", 
           "MasVnrType", "MasVnrArea", "ExterQual", "ExterCond", "Foundation","BsmtQual", 
           "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", 
           "TotalBsmtSF", "Heating", "HeatingQC","CentralAir", "Electrical", "fstFlrSF", "sndFlrSF",
           "LowQualFinSF","GrLivArea", "BsmtFullBath","BsmtHalfBath", "FullBath", "HalfBath", "Bedroom",
           "Kitchen","KitchenQual", "TotRmsAbvGrd","Functional", "Fireplaces", "FireplaceQu", "GarageType", 
           "GarageYrBlt","GarageFinish", "GarageCars", "GarageArea", "GarageQual", "GarageCond", "PavedDrive", 
           "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "tSsnPorch", "ScreenPorch", "PoolArea", "PoolQC", "Fence", 
           "MiscFeature", "MiscVal", "MoSold", "YrSold", "SaleType", "SaleCondition", "SalePrice"]
FEATURES = ["MSSubClass", "MSZoning", "LotFrontage", "LotArea", "Street", "Alley",
           "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope","Neighborhood",
           "Condition1", "Condition2", "BldgType", "HouseStyle", "OverallQual", "OverallCond",
           "YearBuilt", "YearRemodAdd", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", 
           "MasVnrType", "MasVnrArea", "ExterQual", "ExterCond", "Foundation","BsmtQual", 
           "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", 
           "TotalBsmtSF", "Heating", "HeatingQC","CentralAir", "Electrical", "fstFlrSF", "sndFlrSF",
           "LowQualFinSF","GrLivArea", "BsmtFullBath","BsmtHalfBath", "FullBath", "HalfBath", "Bedroom",
           "Kitchen","KitchenQual", "TotRmsAbvGrd","Functional", "Fireplaces", "FireplaceQu", "GarageType", 
           "GarageYrBlt","GarageFinish", "GarageCars", "GarageArea", "GarageQual", "GarageCond", "PavedDrive", 
           "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "tSsnPorch", "ScreenPorch", "PoolArea", "PoolQC", "Fence", 
           "MiscFeature", "MiscVal", "MoSold", "YrSold", "SaleType", "SaleCondition"]
LABEL = "SalesPrice"
CATEGORICAL_COLUMNS = ["MSSubClass", "MSZoning", "Street", "Alley","LotShape", "LandContour", "Utilities", "LotConfig", 
                       "LandSlope","Neighborhood","Condition1", "Condition2", "BldgType", "HouseStyle", "OverallQual", 
                       "OverallCond", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType","ExterQual", 
                       "ExterCond", "Foundation","BsmtQual", 
                       "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",  "Heating", "HeatingQC","CentralAir", "Electrical",
                       "KitchenQual", "Functional","FireplaceQu", "GarageType", "GarageFinish", "GarageCars", "GarageQual",
                       "GarageCond","PavedDrive", "PoolQC", "Fence", "MiscFeature", "MoSold", "YrSold", "SaleType",
                       "SaleCondition"]
                       
        
CONTINUOUS_COLUMNS = ["LotFrontage", "LotArea", "YearBuilt", "YearRemodAdd", "MasVnrArea","BsmtFinSF1", "BsmtFinSF2",
                       "BsmtUnfSF", "TotalBsmtSF", "fstFlrSF", "sndFlrSF",
           "LowQualFinSF","GrLivArea", "BsmtFullBath","BsmtHalfBath", "FullBath", "HalfBath", "Bedroom",
           "Kitchen","TotRmsAbvGrd", "Fireplaces", "GarageYrBlt", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", 
                       "tSsnPorch", "ScreenPorch", "PoolArea", "MiscVal"]


                       
#train = "trainproperformat.csv".sample(frac=0.8,random_state=200)
#test = "trainproperformat.cvs".drop(train.index)
training_set = pd.read_csv("trainproperformat.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS).sample(frac=0.8,random_state=200)
test_set = pd.read_csv("trainproperformat.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS).drop(training_set.index)
prediction_set = pd.read_csv("testproperformat.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)
#print(len(CONTINUOUS_COLUMNS))
#print(test_set)
#print(training_set)

In [7]:
def build_estimator(model_dir):
  #"""Build an estimator."""
  # Sparse base columns.
    MSSubClass= tf.contrib.layers.sparse_column_with_hash_bucket("MSSubClass", hash_bucket_size=1000)
    MSZoning= tf.contrib.layers.sparse_column_with_hash_bucket("MSZoning", hash_bucket_size=1000)
    Street= tf.contrib.layers.sparse_column_with_hash_bucket("Street", hash_bucket_size=1000)
    Alley= tf.contrib.layers.sparse_column_with_hash_bucket("Alley", hash_bucket_size=1000)
    LotShape= tf.contrib.layers.sparse_column_with_hash_bucket("LotShape", hash_bucket_size=1000)
    LandContour= tf.contrib.layers.sparse_column_with_hash_bucket("LandContour", hash_bucket_size=1000)
    Utilities= tf.contrib.layers.sparse_column_with_hash_bucket("Utilities", hash_bucket_size=1000)
    LotConfig= tf.contrib.layers.sparse_column_with_hash_bucket("LotConfig", hash_bucket_size=1000)
    LandSlope= tf.contrib.layers.sparse_column_with_hash_bucket("LandSlope", hash_bucket_size=1000)
    Neighborhood= tf.contrib.layers.sparse_column_with_hash_bucket("Neighborhood", hash_bucket_size=1000)
    Condition1= tf.contrib.layers.sparse_column_with_hash_bucket("Condition1", hash_bucket_size=1000)
    Condition2= tf.contrib.layers.sparse_column_with_hash_bucket("Condition2", hash_bucket_size=1000)
    BldgType= tf.contrib.layers.sparse_column_with_hash_bucket("BldgType", hash_bucket_size=1000)
    HouseStyle= tf.contrib.layers.sparse_column_with_hash_bucket("HouseStyle", hash_bucket_size=1000)
    OverallQual= tf.contrib.layers.sparse_column_with_hash_bucket("OverallQual", hash_bucket_size=1000)
    OverallCond= tf.contrib.layers.sparse_column_with_hash_bucket("OverallCond", hash_bucket_size=1000)
    RoofStyle= tf.contrib.layers.sparse_column_with_hash_bucket("RoofStyle", hash_bucket_size=1000)
    RoofMatl= tf.contrib.layers.sparse_column_with_hash_bucket("RoofMatl", hash_bucket_size=1000)
    Exterior1st= tf.contrib.layers.sparse_column_with_hash_bucket("Exterior1st", hash_bucket_size=1000)
    Exterior2nd= tf.contrib.layers.sparse_column_with_hash_bucket("Exterior2nd", hash_bucket_size=1000)
    MasVnrType= tf.contrib.layers.sparse_column_with_hash_bucket("MasVnrType", hash_bucket_size=1000)
    ExterQual= tf.contrib.layers.sparse_column_with_hash_bucket("ExterQual", hash_bucket_size=1000)
    ExterCond= tf.contrib.layers.sparse_column_with_hash_bucket("ExterCond", hash_bucket_size=1000)
    Foundation= tf.contrib.layers.sparse_column_with_hash_bucket("Foundation", hash_bucket_size=1000)
    BsmtQual= tf.contrib.layers.sparse_column_with_hash_bucket("BsmtQual", hash_bucket_size=1000)
    BsmtCond= tf.contrib.layers.sparse_column_with_hash_bucket("BsmtCond", hash_bucket_size=1000)
    BsmtExposure= tf.contrib.layers.sparse_column_with_hash_bucket("BsmtExposure", hash_bucket_size=1000)
    BsmtFinType1= tf.contrib.layers.sparse_column_with_hash_bucket("BsmtFinType1", hash_bucket_size=1000)
    BsmtFinType2= tf.contrib.layers.sparse_column_with_hash_bucket("BsmtFinType2", hash_bucket_size=1000)
    Heating= tf.contrib.layers.sparse_column_with_hash_bucket("Heating", hash_bucket_size=1000)
    HeatingQC= tf.contrib.layers.sparse_column_with_hash_bucket("HeatingQC", hash_bucket_size=1000)
    CentralAir= tf.contrib.layers.sparse_column_with_hash_bucket("CentralAir", hash_bucket_size=1000)
    Electrical= tf.contrib.layers.sparse_column_with_hash_bucket("Electrical", hash_bucket_size=1000)
    KitchenQual= tf.contrib.layers.sparse_column_with_hash_bucket("KitchenQual", hash_bucket_size=1000)
    Functional= tf.contrib.layers.sparse_column_with_hash_bucket("Functional", hash_bucket_size=1000)
    FireplaceQu= tf.contrib.layers.sparse_column_with_hash_bucket("FireplaceQu", hash_bucket_size=1000)
    GarageType= tf.contrib.layers.sparse_column_with_hash_bucket("GarageType", hash_bucket_size=1000)
    GarageFinish= tf.contrib.layers.sparse_column_with_hash_bucket("GarageFinish", hash_bucket_size=1000)
    GarageCars= tf.contrib.layers.sparse_column_with_hash_bucket("GarageCars", hash_bucket_size=1000)
    GarageQual= tf.contrib.layers.sparse_column_with_hash_bucket("GarageQual", hash_bucket_size=1000)
    GarageCond= tf.contrib.layers.sparse_column_with_hash_bucket("GarageCond", hash_bucket_size=1000)
    PavedDrive= tf.contrib.layers.sparse_column_with_hash_bucket("PavedDrive", hash_bucket_size=1000)
    PoolQC= tf.contrib.layers.sparse_column_with_hash_bucket("PoolQC", hash_bucket_size=1000)
    Fence= tf.contrib.layers.sparse_column_with_hash_bucket("Fence", hash_bucket_size=1000)
    MiscFeature= tf.contrib.layers.sparse_column_with_hash_bucket("MiscFeature", hash_bucket_size=1000)
    MoSold= tf.contrib.layers.sparse_column_with_hash_bucket("MoSold", hash_bucket_size=1000)
    YrSold= tf.contrib.layers.sparse_column_with_hash_bucket("YrSold", hash_bucket_size=1000)
    SaleType= tf.contrib.layers.sparse_column_with_hash_bucket("SaleType", hash_bucket_size=1000)
    SaleCondition= tf.contrib.layers.sparse_column_with_hash_bucket("SaleCondition", hash_bucket_size=1000)
  # Continuous base columns.
    LotFrontage= tf.contrib.layers.real_valued_column("LotFrontage")
    LotArea= tf.contrib.layers.real_valued_column("LotArea")
    YearBuilt= tf.contrib.layers.real_valued_column("YearBuilt")
    YearRemodAdd= tf.contrib.layers.real_valued_column("YearRemodAdd")
    MasVnrArea= tf.contrib.layers.real_valued_column("MasVnrArea")
    BsmtFinSF1= tf.contrib.layers.real_valued_column("BsmtFinSF1")
    BsmtFinSF2= tf.contrib.layers.real_valued_column("BsmtFinSF2")
    BsmtUnfSF= tf.contrib.layers.real_valued_column("BsmtUnfSF")
    TotalBsmtSF= tf.contrib.layers.real_valued_column("TotalBsmtSF")
    fstFlrSF= tf.contrib.layers.real_valued_column("fstFlrSF")
    sndFlrSF= tf.contrib.layers.real_valued_column("sndFlrSF")
    LowQualFinSF= tf.contrib.layers.real_valued_column("LowQualFinSF")
    GrLivArea= tf.contrib.layers.real_valued_column("GrLivArea")
    BsmtFullBath= tf.contrib.layers.real_valued_column("BsmtFullBath")
    BsmtHalfBath= tf.contrib.layers.real_valued_column("BsmtHalfBath")
    FullBath= tf.contrib.layers.real_valued_column("FullBath")
    HalfBath= tf.contrib.layers.real_valued_column("HalfBath")
    Bedroom= tf.contrib.layers.real_valued_column("Bedroom")
    Kitchen= tf.contrib.layers.real_valued_column("Kitchen")
    TotRmsAbvGrd= tf.contrib.layers.real_valued_column("TotRmsAbvGrd")
    Fireplaces= tf.contrib.layers.real_valued_column("Fireplaces")
    GarageYrBlt= tf.contrib.layers.real_valued_column("GarageYrBlt")
    GarageArea= tf.contrib.layers.real_valued_column("GarageArea")
    WoodDeckSF= tf.contrib.layers.real_valued_column("WoodDeckSF")
    OpenPorchSF= tf.contrib.layers.real_valued_column("OpenPorchSF")
    EnclosedPorch= tf.contrib.layers.real_valued_column("EnclosedPorch")
    tSsnPorch= tf.contrib.layers.real_valued_column("tSsnPorch")
    ScreenPorch= tf.contrib.layers.real_valued_column("ScreenPorch")
    PoolArea= tf.contrib.layers.real_valued_column("PoolArea")
    MiscVal= tf.contrib.layers.real_valued_column("MiscVal")
  # Transformations.
  #age_buckets = tf.contrib.layers.bucketized_column(age,
                                                    #boundaries=[
                                                    #    18, 25, 30, 35, 40, 45,
                                                    #    50, 55, 60, 65
                                                    #])

  # Wide columns and deep columns.
    wide_columns = [MSSubClass, MSZoning, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageCars, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, MoSold, YrSold, SaleType, SaleCondition]
    deep_columns = [LotFrontage, LotArea, YearBuilt, YearRemodAdd, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, fstFlrSF, sndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, Bedroom, Kitchen, TotRmsAbvGrd, Fireplaces, GarageYrBlt, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, tSsnPorch, ScreenPorch, PoolArea, MiscVal]
     # tf.contrib.layers.embedding_column(workclass, dimension=8),
      #tf.contrib.layers.embedding_column(education, dimension=8),
      #tf.contrib.layers.embedding_column(gender, dimension=8),
      #tf.contrib.layers.embedding_column(relationship, dimension=8),
      #tf.contrib.layers.embedding_column(native_country,
      #                                   dimension=8),
      #tf.contrib.layers.embedding_column(occupation, dimension=8),
      #age,
      #education_num,
      #capital_gain,
      #capital_loss,
      #hours_per_week,
  #]

    if FLAGS.model_type == "wide":
        m = tf.contrib.learn.LinearClassifier(model_dir=model_dir,
                                          feature_columns=wide_columns)
    elif FLAGS.model_type == "deep":
        m = tf.contrib.learn.DNNClassifier(model_dir=model_dir,
                                       feature_columns=deep_columns,
                                       hidden_units=[100, 50])
    else:
        m = tf.contrib.learn.DNNLinearCombinedClassifier(
            model_dir=model_dir,
            linear_feature_columns=wide_columns,
            dnn_feature_columns=deep_columns,
            dnn_hidden_units=[100, 50])
    return m


In [11]:
def input_fn(df):
    """Input builder function."""
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {
     k: tf.SparseTensor(
            indices=[[i, 0] for i in range(df[k].size)],
            values=df[k].values,
            dense_shape=[df[k].size, 1])
            for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols)
    feature_cols.update(categorical_cols)
  # Converts the label column into a constant Tensor.
    label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
    return feature_cols, label

In [12]:
def train_and_eval():
    """Train and evaluate the model."""
    df_train = training_set
    df_test = test_set
    df_predict = prediction_set
    #df_train[LABEL_COLUMN] = (
    #  df_train["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)
    #df_test[LABEL_COLUMN] = (
    #  df_test["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)

    model_dir = tempfile.mkdtemp() #if not FLAGS.model_dir else FLAGS.model_dir
    print("model directory = %s" % model_dir)

    m = build_estimator(model_dir)
    m.fit(input_fn=lambda: input_fn(df_train), steps=FLAGS.train_steps)
    results = m.evaluate(input_fn=lambda: input_fn(df_test), steps=1)
    for key in sorted(results):
        print("%s: %s" % (key, results[key]))


In [13]:
def main(_):
    train_and_eval()


if __name__ == "__main__":
    tf.app.run()

model directory = /tmp/tmp96qfi2w1


TypeError: __init__() got an unexpected keyword argument 'dense_shape'